Contents 
1. Import Libraries
2. Group df by department_id and aggregate by order_number mean 
3. Find differences in statistics between subset and entire df 
4. Create loyalty flag 
5. Check statistics on Loyalty Flags
6. Create spending flag 
7. Create frequent customer flag 
8. Export df as pickle 

In [1]:
#import libraries 
import pandas as pd
import numpy as np
import os

In [2]:
#define path 
path = '/Users/wenboyang/Desktop/Data Analytics/05-08-2023 Instacart Basket Analysis /02 Data/'

In [3]:
#import orders_products_merged_updated.pkl
df = pd.read_pickle(os.path.join(path, 'Prepared Data', 'orders_products_merged_updated.pkl'))

Step 2

In [4]:
#group by department_id and aggregate by order_number mean
df.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


Step 3

The min and max of the entire dataframe are 15 and 22 whereas the min and max of the subset are 11 and 19. 

Step 4 Loyalty Flag

In [5]:
#create 'max_order' column 
df['max_order'] = df.groupby(['user_id'])['order_number'].transform(np.max)

In [6]:
#create loyalty flags
df.loc[df['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
df.loc[(df['max_order'] <= 40) & (df['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
df.loc[df['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [7]:
df.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest day,busiest_days,busiest_period_of_day,max_order,loyalty_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly days,Average orders,10,New customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Average orders,10,New customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Most orders,10,New customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Slowest days,Average orders,10,New customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Slowest days,Most orders,10,New customer


Step 5

The marketing team at Instacart wants to know whether there’s a difference between the spending habits of the three types of customers you identified. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer). What you’re trying to determine is whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers.

In [8]:
#check basic spending statistics via loyalty flag 
df.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

New customers and regular customers buy more expensive items than loyal customers 

Step 6: If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”
If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”

In [9]:
#create mean price column 
df['mean_price']=df.groupby(['user_id'])['order_number'].transform(np.mean)

In [10]:
#check new column 
df.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,department_id,prices,_merge,price_range_loc,busiest day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_price
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,7,9.0,both,Mid-range product,Regularly busy,Regularly days,Average orders,10,New customer,5.813559
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Average orders,10,New customer,5.813559
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,7,9.0,both,Mid-range product,Regularly busy,Slowest days,Most orders,10,New customer,5.813559
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,7,9.0,both,Mid-range product,Least busy,Slowest days,Average orders,10,New customer,5.813559
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,7,9.0,both,Mid-range product,Least busy,Slowest days,Most orders,10,New customer,5.813559


In [11]:
#create spending flags
df.loc[df['mean_price'] >= 10, 'spending_flag'] = 'High spender'
df.loc[df['mean_price'] <= 10,'spending_flag'] = 'Low spender'

In [12]:
#check spending_flag column 
df.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,prices,_merge,price_range_loc,busiest day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_price,spending_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,9.0,both,Mid-range product,Regularly busy,Regularly days,Average orders,10,New customer,5.813559,Low spender
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,9.0,both,Mid-range product,Regularly busy,Slowest days,Average orders,10,New customer,5.813559,Low spender
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,9.0,both,Mid-range product,Regularly busy,Slowest days,Most orders,10,New customer,5.813559,Low spender
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,9.0,both,Mid-range product,Least busy,Slowest days,Average orders,10,New customer,5.813559,Low spender
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,9.0,both,Mid-range product,Least busy,Slowest days,Most orders,10,New customer,5.813559,Low spender


Step 7: Frequent customer flag 

In [13]:
#check user order averages 
df.groupby(['user_id'])['days_since_prior_order'].transform(np.mean)

0           20.259259
1           20.259259
2           20.259259
3           20.259259
4           20.259259
              ...    
32404854    10.380062
32404855    10.380062
32404856    22.058824
32404857    22.058824
32404858    14.372727
Name: days_since_prior_order, Length: 32404859, dtype: float64

In [14]:
#create new column 'mean_freq' by getting mean of orders grouped by 'user_id'
df['mean_freq']=df.groupby(['user_id'])['days_since_prior_order'].transform(np.mean)

In [15]:
#create frequency flags 
df.loc[df['mean_freq'] > 20, 'freq_flag'] = 'Non-frequent customer'
df.loc[(df['mean_freq'] >10) & (df['mean_freq'] <= 20),'freq_flag'] = 'Regular customer'
df.loc[df['mean_freq']<=10, 'freq_flag'] = 'Frequent customer'

In [16]:
#check results
df.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,price_range_loc,busiest day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_price,spending_flag,mean_freq,freq_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Mid-range product,Regularly busy,Regularly days,Average orders,10,New customer,5.813559,Low spender,20.259259,Non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Mid-range product,Regularly busy,Slowest days,Average orders,10,New customer,5.813559,Low spender,20.259259,Non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Mid-range product,Regularly busy,Slowest days,Most orders,10,New customer,5.813559,Low spender,20.259259,Non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Mid-range product,Least busy,Slowest days,Average orders,10,New customer,5.813559,Low spender,20.259259,Non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Mid-range product,Least busy,Slowest days,Most orders,10,New customer,5.813559,Low spender,20.259259,Non-frequent customer


Step 8: Export in pickle format

In [17]:
df.to_pickle(os.path.join(path, 'Prepared Data', 'orders_products_merged_updated_2.pkl'))